In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/Users/yinchiahuang/Library/Mobile Documents/com~apple~CloudDocs/CS677/Assignments/yinchia@bu.edu_hw1/HMC_Honda Motor Co., Ltd.csv')

In [3]:
# calculate shares and profit
def buy(investment,price):
    shares = investment / price
    return shares


def sell(cash,shares, price): # cash = cash in hand (not including stock value)
    revenue = shares * price
    new_cash = revenue - cash
    return new_cash

### Momentum ( moving average)

In [4]:
def get_ma_df(df,win_st,win_lt):
    ma_df = df.copy()
    ma_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
    ma_df = ma_df[['Date', 'Price']]
    ma_df['ma_st'] = ma_df['Price'].rolling(win_st).mean()
    ma_df['ma_lt'] = ma_df['Price'].rolling(win_lt).mean()
    #ma_df = ma_df.tail(len(ma_df)-win_lt)
    ma_df['ma_buy'] = ma_df['ma_st'] > ma_df['ma_lt']
    ma_df['ma_sell'] = ma_df['ma_st'] < ma_df['ma_lt']
    ma_df['ma_transactions'] = np.where(ma_df['ma_buy'] != ma_df['ma_buy'].shift(+1),1,0)
    ma_df
    columnsTitles = ['Date','Price','ma_buy','ma_sell','ma_transactions']
    ma_df = ma_df.reindex(columns=columnsTitles)
    
    # make sure all the transactions start at buy and end at sell
    transaction_start_index = ma_df[(ma_df['ma_transactions']==1) & (ma_df['ma_buy']==True)].index.values.astype(int)
    np.array(transaction_start_index).tolist()
    transaction_start_index = transaction_start_index[0]

    transaction_end_index = ma_df[(ma_df['ma_transactions']==1) & (ma_df['ma_sell']==True)].index.values.astype(int)
    np.array(transaction_end_index).tolist()
    transaction_end_index = transaction_end_index[-1]


    for row in range(len(ma_df)):
        if (row < transaction_start_index)|(row > transaction_end_index):
            ma_df.at[row,'ma_transactions'] =0
    
    ma_df = ma_df.reset_index()
    initial_capital = 100
    cash = []
    shares = []
    
    for row in range(len(ma_df)):
        # buy: shares + ; cash = 0 
        if (ma_df.loc[row]['ma_buy'] == True) and (ma_df.loc[row]['ma_transactions'] ==1): 
            cash.append(0)
            if row == 0 :
                shares.append(buy(initial_capital,ma_df.loc[row]['Price']))
            else:
                shares.append(buy(cash[row-1],ma_df.loc[row]['Price']))
        # sell : shares - ; cash +/-
        elif (ma_df.loc[row]['ma_sell'] == True) and (ma_df.loc[row]['ma_transactions'] ==1):
            shares.append(0)
            cash.append(sell(cash[row-1],shares[row-1],ma_df.loc[row]['Price'])) 
        # hold : shares and cash no change
        else:
            if row == 0:
                shares.append(0)
                cash.append(initial_capital)
            else:
                shares.append(shares[row-1])
                cash.append(cash[row-1])
        
    ma_df['ma_shares']= shares
    ma_df['ma_cash'] = cash
    
    return ma_df




In [5]:
#ma_df = get_ma_df(df, 14, 50)

In [6]:
#ma_df


### Bollinger Band

In [7]:
def get_bb_df(df, k,win):
    bb_df = df.copy()
    bb_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
    bb_df = bb_df[['Date', 'Price']]
    bb_df['bb_ma'] = bb_df['Price'].rolling(win).mean()
    bb_df['bb_std'] = bb_df['Price'].rolling(win).std()        
    bb_df['lower_band'] = bb_df['bb_ma']- k* bb_df['bb_std']
    bb_df['upper_band'] = bb_df['bb_ma'] + k* bb_df['bb_std']
    bb_df['bb_sell'] = bb_df['Price'] < bb_df['lower_band']
    bb_df['bb_buy'] = bb_df['Price'] > bb_df['upper_band']
        

    conditions = [(bb_df['bb_buy']== True) & (bb_df['bb_sell'] == False), (bb_df['bb_buy']== False) & (bb_df['bb_sell'] == True),(bb_df['bb_buy']== False) & (bb_df['bb_sell'] == False)]
    choices = [1,1,0]
    bb_df['bb_suppose_trans'] = np.select(conditions,choices)
    
    bb_temp_df = bb_df[bb_df['bb_suppose_trans']==1]
    bb_temp_df['bb_transactions'] = np.where(bb_temp_df['bb_buy'] !=bb_temp_df['bb_buy'].shift(+1),1,0)

    transactions_index = bb_temp_df[bb_temp_df['bb_transactions']==1].index.values.astype(int)
    np.array(transactions_index).tolist()
    
    transactions = []

    for row in range(len(bb_df)):
        if row in transactions_index:
            transactions.append(1)
        else:
            transactions.append(0)
        
    bb_df['bb_transactions'] = transactions 

    # make sure all the transactions start at buy and end at sell
    transaction_start_index = bb_df[(bb_df['bb_transactions']==1) & (bb_df['bb_buy']==True)].index.values.astype(int)
    np.array(transaction_start_index).tolist()
    transaction_start_index = transaction_start_index[0]
    
    transaction_end_index = bb_df[(bb_df['bb_transactions']==1) & (bb_df['bb_sell']==True)].index.values.astype(int)
    np.array(transaction_end_index).tolist()
    transaction_end_index = transaction_end_index[-1]
    

    for row in range(len(bb_df)):
        if (row < transaction_start_index)|(row > transaction_end_index):
            bb_df.at[row,'bb_transactions'] =0
            
    columnsTitles = ['Date','Price','bb_buy','bb_sell','bb_transactions']
    bb_df = bb_df[columnsTitles]
    
    bb_df = bb_df.reset_index()
    initial_capital = 100
    cash = []
    shares = []
    
    for row in range(len(bb_df)):
        # buy: shares + ; cash = 0 
        if (bb_df.loc[row]['bb_buy'] == True) and (bb_df.loc[row]['bb_transactions'] ==1): 
            cash.append(0)
            if row == 0 :
                shares.append(buy(initial_capital,bb_df.loc[row]['Price']))
            else:
                shares.append(buy(cash[row-1],bb_df.loc[row]['Price']))
        # sell : shares - ; cash +/-
        elif (bb_df.loc[row]['bb_sell'] == True) and (bb_df.loc[row]['bb_transactions'] ==1):
            shares.append(0)
            cash.append(sell(cash[row-1],shares[row-1],bb_df.loc[row]['Price'])) 
        # hold : shares and cash no change
        else:
            if row ==0:
                shares.append(0)
                cash.append(initial_capital)    
            else:
                shares.append(shares[row-1])
                cash.append(cash[row-1])
        
    bb_df['bb_shares']= shares
    bb_df['bb_cash'] = cash
    
    return bb_df




In [8]:
#bb_df = get_bb_df(df, 3,56)
#bb_df

### Consecutive days

In [9]:
def get_con_df(df,w):
    con_df = compute_sell_buy_df(df,w)
    con_df = con_df.reset_index()
    initial_capital = 100
    cash = []
    shares = []
    
    for row in range(len(con_df)):
        # buy: shares + ; cash = 0 
        if (con_df.loc[row]['con_buy/sell'] == 'buy'): 
            cash.append(0)
            if row == 0 :
                shares.append(buy(initial_capital,bb_df.loc[row]['Price']))
            else:
                shares.append(buy(cash[row-1],con_df.loc[row]['Price']))
        # sell : shares - ; cash +/-
        elif (con_df.loc[row]['con_buy/sell'] == 'sell'):
            shares.append(0)
            cash.append(sell(cash[row-1],shares[row-1],con_df.loc[row]['Price'])) 
        # hold : shares and cash no change
        else:
            if row ==0:
                shares.append(0)
                cash.append(initial_capital)    
            else:
                shares.append(shares[row-1])
                cash.append(cash[row-1])
        
    con_df['con_shares']= shares
    con_df['con_cash'] = cash
    
    columnsTitles = ['Date','Price','con_buy/sell','con_shares','con_cash']
    con_df = con_df.reindex(columns=columnsTitles)
    
    return con_df


def compute_percent_df(df):
    try:
        new_df = df.copy()
        new_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
        new_df = new_df[['Date', 'Price']]
        new_df['price_change(%)'] = new_df['Price'].pct_change() * 100
        new_df.fillna(0, inplace = True)
        return new_df
    except Exception as e:
        print(e)
        return None 


def compute_sell_buy_df(df,w):
    new_df = compute_percent_df(df)
    new_df['good/bad(1/0)'] = np.where(new_df['price_change(%)'] >= 0,1,0)
    
    count = 0
    list = new_df['good/bad(1/0)']
    buy_list = [] # index of row to buy
    sell_list = [] # index or row to sell
    buy_sell_list = []

    for i in range(len(list)):
        if list[i] == 0:
            count +=1
            if count == w:
                count = 0 
                buy_list.append(i)
                sell_list.append(i+1)
                i+=2
            else:
                i+=1
        else:
            count = 0
            i+=1
    
        
    for i in range(len(df)):
        if i in buy_list:
            buy_sell_list.append('buy')
        elif i in sell_list:
            buy_sell_list.append('sell')
        else:
            buy_sell_list.append('hold')
            
    
    new_df['con_buy/sell'] = buy_sell_list
    
    return new_df



In [10]:
#con_df = get_con_df(df,3) 
#con_df

### naïve investment strategy: assume that tomorrow will be the same as today:
(a). if the price of the stock today is higher than yesterday, assume that it will be higher tomorrow\n
(b). it the price of stock today is lower than yesterday, it will be lower tomorrow

In [11]:
def get_na_df():
    na_df = df.copy()
    na_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
    na_df = na_df[['Date', 'Price']]
    na_df['daily_returns'] = (na_df.Price - na_df.Price.shift(1))/na_df.Price.shift(1)
    na_df['buy/sell_signal']= np.where(na_df['daily_returns']>=0,'buy','sell')
    
    # make sure all the transactions start at buy and end at sell
    transaction_start_index = na_df[na_df['buy/sell_signal']== 'buy'].index.values.astype(int)
    np.array(transaction_start_index).tolist()
    transaction_start_index = transaction_start_index[0]
    
    transaction_end_index = na_df[na_df['buy/sell_signal']== 'sell'].index.values.astype(int)
    np.array(transaction_end_index).tolist()
    transaction_end_index = transaction_end_index[-1]
    

    for row in range(len(na_df)):
        if (row < transaction_start_index)|(row > transaction_end_index):
            na_df.at[row,'buy/sell_signal'] = 'hold'
            
    # cannot buy consecutively or sell consecutively
    real_transactions = na_df['buy/sell_signal'].copy()

    current = 'buy'
    for i in range(transaction_start_index+1,len(real_transactions)):

        if (real_transactions[i] == current) and (real_transactions[i] !='hold'):
            real_transactions[i] = 'hold'
        if (real_transactions[i] != current) and (real_transactions[i] !='hold'):
            current = real_transactions[i]
            
    na_df['na_buy/sell']=real_transactions
    
    # Calculate the shares and profit
    initial_capital = 100
    cash = []
    shares = []

    for row in range(len(na_df)):
        # buy: shares + ; cash = 0 
        if (na_df.loc[row]['na_buy/sell'] == 'buy'): 
            cash.append(0)
            if row == 0 :
                shares.append(buy(initial_capital,na_df.loc[row]['Price']))
            else:
                shares.append(buy(cash[row-1],na_df.loc[row]['Price']))
        # sell : shares - ; cash +/-
        elif (na_df.loc[row]['na_buy/sell'] == 'sell'):
            shares.append(0)
            cash.append(sell(cash[row-1],shares[row-1],na_df.loc[row]['Price'])) 
        # hold : shares and cash no change
        else:
            if row ==0:
                shares.append(0)
                cash.append(initial_capital)    
            else:
                shares.append(shares[row-1])
                cash.append(cash[row-1])

    na_df['na_shares']= shares
    na_df['na_cash'] = cash
    
    return na_df


In [12]:
na_df = get_na_df()
na_df

,Date,Price,daily_returns,buy/sell_signal,na_buy/sell,na_shares,na_cash
0,2013/9/16,34.026104,NaN,hold,hold,0.000000,100.000000
1,2013/9/17,33.885891,-0.004121,hold,hold,0.000000,100.000000
2,2013/9/18,34.876091,0.029222,buy,buy,2.867294,0.000000
3,2013/9/19,34.692081,-0.005276,sell,sell,0.000000,99.472389
4,2013/9/20,34.087433,-0.017429,sell,hold,0.000000,99.472389
5,2013/9/23,34.104969,0.000514,buy,buy,2.916654,0.000000
6,2013/9/24,34.140015,0.001028,buy,hold,2.916654,0.000000
7,2013/9/25,34.135738,-0.000125,sell,sell,0.000000,99.562132
8,2013/9/26,34.514431,0.011094,buy,buy,2.884652,0.000000
9,2013/9/27,34.100506,-0.011993,sell,sell,0.000000,98.368102


### Run these there strategies assuming we start with 100 dollars. Compare the results of these 75 combinations.

In [13]:
ma_windows_short = [28,28,14,14,13]
ma_windows_long = [63,56,70,63,56]
bb_ks = [3,2.5,2.75,2.5,2]
bb_ws = [56,84,63,98,77]
con_ws = [2,3,4]

ma_s_win = []
ma_l_win = []
ma_P_L = []

bb_k = []
bb_w = []
bb_P_L = []

con_w =[]
con_P_L = []



for i in range(5):
    for j in range(5):
        for k in range(3):
            ma_df = get_ma_df(df,ma_windows_short[i],ma_windows_long[j])
            ma_profit = ma_df.loc[len(ma_df)-1]['ma_cash'] 
            bb_df = get_bb_df(df,bb_ks[i],bb_ws[j])
            bb_profit = bb_df.loc[len(bb_df)-1]['bb_cash']
            con_df = get_con_df(df,con_ws[k])
            con_profit = con_df.loc[len(con_df)-1]['con_cash']

            ma_s_win.append(ma_windows_short[i])
            ma_l_win.append(ma_windows_long[i])
            ma_P_L.append(ma_profit)
            
            bb_k.append(bb_ks[i])
            bb_w.append(bb_ws[j])
            bb_P_L.append(bb_profit)
            
            con_w.append(con_ws[k])
            con_P_L.append(con_profit)

            
d = {'ma_s_win': ma_s_win, 'ma_l_win': ma_l_win, 'ma_P/L':ma_P_L,'bb_k': bb_k,'bb_w':bb_w,'bb_P/L':bb_P_L, 'con_w':con_w,'con_P/L':con_P_L}
df_result = pd.DataFrame(data=d)      
            
df_result
            

/Users/yinchiahuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ma_s_win,ma_l_win,ma_P/L,bb_k,bb_w,bb_P/L,con_w,con_P/L
0,28,63,115.286066,3.0,56,117.353168,2,91.869246
1,28,63,115.286066,3.0,56,117.353168,3,96.358152
2,28,63,115.286066,3.0,56,117.353168,4,102.390864
3,28,63,115.243337,3.0,84,100.000000,2,91.869246
4,28,63,115.243337,3.0,84,100.000000,3,96.358152
5,28,63,115.243337,3.0,84,100.000000,4,102.390864
6,28,63,92.906498,3.0,63,100.000000,2,91.869246
7,28,63,92.906498,3.0,63,100.000000,3,96.358152
8,28,63,92.906498,3.0,63,100.000000,4,102.390864
9,28,63,115.286066,3.0,98,100.000000,2,91.869246


In [14]:
df_result.to_csv('Compare_3_strategies.csv')

### Conclusion:
Q: How does this naïve strategy compare with the other strategies you considered?
A: Naïve investment strategy has the worst performance compared to momentum, Bollinger Band, and consecutive days investment strategies. The remaining money of this strategy is $79.6 if we put $100 in the very beginning. However, the other three strategies have at aleast $90 in the end.  
